# 3. Gradient Descent for Regression

### Imports

In [47]:
import numpy as np
import pandas as pd
import itertools
from collections import deque
import matplotlib.pyplot as plt

### Global

In [2]:
np.random.seed = 12345
datasets_path = "../Datasets/"

### Variables

In [3]:
# Storing data in dataframes, and dropping a random column with unknown values in it, plus renaming the columns
df_train = pd.read_csv(datasets_path + 
                       "Dataset_2_train.csv", 
                       header = None).drop(labels=2, axis=1).rename(index=str, columns={0: "x", 1: "y"})
df_test = pd.read_csv(datasets_path + 
                      "Dataset_2_test.csv", 
                      header = None).drop(labels=2, axis=1).rename(index=str, columns={0: "x", 1: "y"})
df_val = pd.read_csv(datasets_path + 
                     "Dataset_2_valid.csv", 
                     header = None).drop(labels=2, axis=1).rename(index=str, columns={0: "x", 1: "y"})

## Part 1

In [61]:
regression_fn_coeffs = np.random.rand(2)*10
patience = 1000  # To measure learning stature
print(np.poly1d(regression_fn_coeffs))

 
3.318 x + 3.392


In [62]:
def get_loss(xs, ys, w_i):
    loss = 0
    for x, y in zip(xs,ys):
        loss += (np.polyval(regression_fn_coeffs, x) - y) * (x**(1-w_i))
    return loss/(len(xs))

def compute_mse(xs, ys):
    mse = ((np.polyval(regression_fn_coeffs, xs) - ys) ** 2).mean()
    return mse

def plot_fcts():
    plt.plot(df_train.x.values, np.polyval(regression_fn_coeffs, df_train.x.values), 'ro',
             df_train.x.values, df_train.y.values, 'bo')
    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)
    plt.show()

In [65]:
def train(df_train, df_val, learning_rate = 1e-6):
    # train and valication data
    xs, ys = df_train.x.values, df_train.y.values
    x_val, y_val = df_val.x.values, df_val.y.values
    
    # rolling window over MSE to stop when stature
    rolling_loss_val = deque(maxlen=patience)
    all_mse_val, all_mse_tr = [], []
    epoch = 0
    
    # Training loop, stoping based on rolling_mse
    while True:
        loss = []
        for c_i, c in enumerate(regression_fn_coeffs):
            loss.append(get_loss(xs, ys, c_i))
            # update weights
            regression_fn_coeffs[c_i] -= learning_rate * loss[-1]

        # get epoch MSE for validation and train
        current_mse_val, current_mse_tr = compute_mse(x_val, y_val), compute_mse(xs, ys)
        
        # add MSEs and loss to lists
        all_mse_val.append(current_mse_val)
        all_mse_tr.append(current_mse_tr)
        rolling_loss_val.append(np.mean(loss))
        
        # evaluate model every 1000 steps and stop if stature
        if epoch % patience == 0 and epoch >= patience:
            first_half = np.mean(list(itertools.islice(rolling_loss_val, 0, int(patience/2))))
            second_half = np.mean(list(itertools.islice(rolling_loss_val, int(patience/2), patience)))
            if first_half > second_half:
                # Not learning anymore, returning all mse
                return all_mse_tr, all_mse_val
        epoch += 1

In [ ]:
all_mse_train, all_mse_val = train(df_train, df_val, 1e-6)

In [ ]:
# Blue line is validation data
# Red line is training data
plt.plot(range(len(all_mse_train)), all_mse_train, 'ro', range(len(all_mse_val)), all_mse_val, 'bo')
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

### Part 2

In [ ]:
print(np.poly1d(regression_fn_coeffs))

## Part 3